In [1]:
import pymorphy2
import re
from collections import defaultdict
from math import floor


In [2]:
class Question_analizier:
    morph = pymorphy2.MorphAnalyzer ()
    questions = []
    all_words = defaultdict(lambda: 0)
    number_of_words = 0
    
    def printf (self):
        print ('questions\n', self.questions, '\n')
    
    def get_set (self, question):
        words = re.findall(r'[^\s!,.?":;0-9]+', question)
        ret_set = set ()
        
        for word in words:
            word_type = self.morph.parse (word)[0].tag.POS
            #теперь не добавляется всякий мусор
            if word_type != 'INTJ' and word_type != 'PRCL' and word_type != 'CONJ' and word_type != 'PREP' and word_type != 'NPRO':
                ret_set.add (self.morph.parse (word)[0].normal_form) 
                self.all_words [word] += 1
                self.number_of_words += 1
            
        return ret_set        
    
    def add_question (self, question):
        self.questions.append (set())
        self.questions [-1] = self.get_set (question)
        
    def expend_question (self, question_id, another_question):
        set_to_add = self.get_set (another_question)
        self.questions [question_id] = self.questions [question_id] | set_to_add
        
    def get_distance (self, question_1, question_2):
        score = 0;        
        for word_in_common in (question_1 & question_2):
            score += self.number_of_words - self.all_words [word_in_common] #чем реже слово, тем оно важнее. Другой вариант 1/self.all_words [word_in_common]
        
        #all_words = len (question_1 | question_2)        
        #all_words += 1 #чтобы не делить на 0
        
        return 1/(score+1)
        
    def get_closest_question (self, question):
        words = self.get_set (question)
        least_distance_1 = float ("inf")
        least_id_1 = -1
        
        least_distance_2 = float ("inf")
        least_id_2 = -1
        
        least_distance_3 = float ("inf")
        least_id_3 = -1
        
        for i in range (len(self.questions)):
            
            distance = self.get_distance (words, self.questions [i])
            
            if distance < least_distance_1:
                least_distance_2 = least_distance_1
                least_id_2 = least_id_1
                
                least_distance_1 = distance
                least_id_1 = i
            elif distance < least_distance_2:
                least_distance_3 = least_distance_2
                least_id_3 = least_id_2
                
                least_distance_2 = distance
                least_id_2 = i                
            elif distance < least_distance_3:
                least_distance_3 = distance
                least_id_3 = i
                
            print ((least_id_1, least_id_2, least_id_3, self.questions [i]))
                
        return least_id_1, least_id_2, least_id_3
    
    def get_popular_themes (self, number):
        return sorted(self.all_words.items(), key=lambda item:item[1])[-number:]

In [3]:
class Question_interpriter:
    edited_questions = Question_analizier ()
    questions = []
    answers = []
    
    def add (self, question, answer):
        self.edited_questions.add_question (question + ' ' + answer)
        self.questions.append (question)
        self.answers.append (answer)
        
    def get_answer (self, question):
        id_1, id_2, id_3 = self.edited_questions.get_closest_question (question)
        return [self.answers [id_1], self.answers [id_2], self.answers [id_3]]
            
    def expend_question (self, question_id, question):
        self.edited_questions.expend_question (question_id, question)
        
    def printf (self):
        print ('questions\n')
        print (self.questions)
        print ('\nanswers\n')
        print (self.answers)
        self.edited_questions.printf ()
    
    def __init__ (self, path):
        counter = 0
        file = open (path, 'r')
        
        for line in file:
            if not line.isspace():
                
                if counter % 2 == 0:
                    self.edited_questions.add_question (line)
                    self.questions.append (line)
                else:                    
                    self.edited_questions.expend_question(floor (counter/2), line)
                    self.answers.append (line)
                
                counter += 1
        
        file.close
            

In [4]:
GoToHelp = Question_interpriter ('cases.txt')

In [5]:
import telebot
token = '469148892:AAFTem8u1mydH0SXjPRTU8RmpU4R15lVSWc'
password = '2718281828459045'
start_message = 'Здравствуйте! Я бот, который поможет вам найти ответы на вопросы по школе программирования GoTo.'
help_message = 'Чтобы задать вопрос, просто напишите боту, что вы хотели узнать.' + \
               'Чтобы добавить вопрос в базу данных, вы должны знать пароль. Нажмите кнопку "/добавить_вопрос", введите пароль, введите вопрос и на следующей строке ответ'
bot = telebot.TeleBot (token)
admins_id = set ()
command_set = {}
expend_question = ''
expend_question_id = -1


In [6]:
@bot.message_handler (commands = ['start'])
def handle_start (message):
    bot.send_message (message.from_user.id, start_message)

In [7]:
@bot.message_handler (commands = ['help'])
def handle_help (message):
    bot.send_message (message.from_user.id, help_message)    

In [8]:
@bot.message_handler (commands = ['add_question'])
def handle_add_question (message):
    global command_set
    
    if message.from_user.id in admins_id:
        command_set [message.from_user.id] = 'вводит вопрос'
        bot.send_message (message.from_user.id, 'введите вопрос и на следующей строке ответ')       
    else:        
        command_set [message.from_user.id] = 'вводит пароль'
        bot.send_message (message.from_user.id, 'введите пароль')
        
    

In [9]:
@bot.message_handler (content_types = ['text'])
def handle_text (message):
    global commad_set
    global admins_id
    global GoToHelp
    
    print (command_set.get (message.from_user.id))
    print (message.text)
    
    if command_set.get (message.from_user.id) == 'вводит пароль':
        if message.text == password:
            admins_id.add (message.from_user.id)
            handle_add_question (message)
        else:
            command_set [message.from_user.id] == ''
            bot.send_message (message.from_user.id, 'пароль неверный')
    elif command_set.get (message.from_user.id) == 'вводит вопрос':
        command_set [message.from_user.id] = ''
        GoToHelp.add (message.text.splitlines()[0], message.text.splitlines()[1])
        bot.send_message (message.from_user.id, 'вопрос добавлен в')
            
    else:
        answers = GoToHelp.get_answer (message.text)
        bot.send_message (message.from_user.id, '1)' + answers [0])
        bot.send_message (message.from_user.id, '2)' + answers [1])
        bot.send_message (message.from_user.id, '3)' + answers [2])

In [ ]:
bot.polling(none_stop=True) 

вводит пароль
2718281828459045
enter password
ok
вводит вопрос
Как мне добавить словарь синонимов
Нужно парсить сайты с помощью beautifulsoap

Как мне найти синонимы
(0, -1, -1, {'плохой', 'доктор', 'самочувствие', '*********', 'номер', 'делать', 'плохо', 'чувсвый', 'навести'})
(0, 1, -1, {'другой', 'нравиться', 'комната', 'хотеть', 'кома', 'директор', 'мочь', 'твой', 'тот', 'лагерь', 'бесячий', 'переехать', 'нужно', 'сосед', 'договориться'})
(0, 1, 2, {'хорошо', 'gotocoin', 'готокоина', 'тот', 'получить', 'нужно', 'майнить', 'участвовать', 'различный', 'нельзя', 'задание', 'пара', 'конкурс', 'выполнять'})
(0, 1, 2, {'где', 'находиться', 'купить', 'база', 'головинк', 'магазин', 'мороженный', 'выход', 'отдых'})
(0, 1, 2, {'один', 'день', 'хотеть', 'свой', 'тестирование', 'интересный', 'желание', 'написать', 'попасть', 'хорошо', 'сообщить', 'тот', 'направление', 'нужно', 'собеседование', 'потом', 'который'})
(0, 1, 2, {'где', 'кофе', 'лагерь', 'купить', 'нельзя', 'чипсы', 'сделать', 'вре

In [1]:
from bs4 import BeautifulSoup